# Quiz Input

In [1]:
# Class representing a type of question
class Question:
    def __init__(self, nOccurences, topic, topicDesc, goal, learningObjective, difficulty, targetAudience, nOptions, nCorrect, allocatedTime):
        self.nOccurences = nOccurences
        self.topic = topic
        self.topicDesc = topicDesc
        self.goal = goal
        self.learningObjective = learningObjective
        self.difficulty = difficulty
        self.targetAudience = targetAudience
        self.nOptions = nOptions
        self.nCorrect = nCorrect
        self.allocatedTime = allocatedTime
        
    def __repr__(self):
        return f"Question(Occurences: {self.nOccurences}, Topic: {self.topic}, Difficulty: {self.difficulty}, Options: {self.nOptions}, Correct: {self.nCorrect})"

    def equals(self, question):
        # Check if all attributes are equal
        return (
            self.topic == question.topic and
            self.topicDesc == question.topicDesc and
            self.goal == question.goal and
            self.learningObjective == question.learningObjective and
            self.difficulty == question.difficulty and
            self.targetAudience == question.targetAudience and
            self.nOptions == question.nOptions and
            self.nCorrect == question.nCorrect and
            self.allocatedTime == question.allocatedTime
        )

In [2]:
# Class that contains and manages the question types
class QuizManager:
    def __init__(self):
        self.questions = []

    # adds a question type to the list of questions
    def addQuestion(self, question, count=1):
        for q in self.questions:
            if question.equals(q):
                q.nOccurences += question.nOccurences
                return
        self.questions.append(question)

    # removes a question type from the list of questions based on its index
    def removeQuestion(self, index):
        if index < len(self.questions):
            del self.questions[index]
        else:
            print("Invalid question index.")

    # lists all the added question types
    def listQuestions(self):
        for idx, question in enumerate(self.questions):
            print(f"{idx}: {question}")


In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Creates the Jupyter GUI to interact with (add, remove, list) the questions
def interactWithQuizManager(quizManager):
    #creates an visual output to display the list of question
    questionListOutput = widgets.Output()

    # callback function for when the add button is clicked
    def addQuestion(button):
        with questionListOutput:
            clear_output()
            # creates a question type based on the values of the input
            question = Question(int(countInput.value),topicInput.value, topicDescInput.value, goalInput.value,
                                learningObjectiveInput.value, difficultyInput.value,
                                targetAudienceInput.value, int(nOptionsInput.value),
                                int(nCorrectInput.value), allocatedTimeInput.value)
            # adds that question to the quiz manager
            quizManager.addQuestion(question)
            #updates the index input
            indexInput.max=len(quizManager.questions)-1
            print("Question added successfully.")
            
            # updates the displayed list of questions
            quizManager.listQuestions()

    # callback function for when the remove button is clicked
    def removeQuestion(button):
        with questionListOutput:
            clear_output()
            try:
                # removes question type based on its index (value of the index input)
                index = int(indexInput.value)
                quizManager.removeQuestion(index)
                print(f"Removed question at index {index}.")
            except ValueError:
                print("Please enter a valid integer for the question index.")
            except IndexError:
                print("Invalid index. Please enter a correct question index.")
            quizManager.listQuestions()

    # style of the labels of the inputs
    st = {'description_width': 'initial'}
    # Inputs
    topicInput = widgets.Text(description="Topic:", style=st)
    topicDescInput = widgets.Textarea(description="Topic Description:", style=st)
    goalInput = widgets.Dropdown(description="Level of Bloom's taxonomy", 
                                  options=["Remember", "Understand", "Apply","Analyze","Evaluate","Create"], value="Understand", style=st)
    learningObjectiveInput = widgets.Text(description="Learning Objective:", style=st)
    difficultyInput = widgets.Dropdown(description="Difficulty of the MCQ", 
                                        options=["Beginner", "Intermediate", "Advanced"], value="Intermediate", style=st)
    targetAudienceInput = widgets.Textarea(description="Audience:", style=st)
    nOptionsInput = widgets.IntSlider(description="Number of options per question:", value=4,min=2,max=6, style=st)
    nCorrectInput = widgets.IntSlider(description="Number of correct answers per question:", value=1,min=1,max=6, style=st)
    allocatedTimeInput = widgets.Text(description="Allocated time:", style=st)

    # Add button inputs
    countInput = widgets.IntSlider(min=1, max=20, value=1, description="Number of Questions to add:", style=st)
    addButton = widgets.Button(description="Add Question")
    addButton.on_click(addQuestion)

    # Remove button inputs
    indexInput = widgets.IntSlider(min=0, max=min(0,len(quiz_manager.questions)), value=0, description="Remove Index:", style=st)
    removeButton = widgets.Button(description="Remove Question")
    removeButton.on_click(removeQuestion)

    # layout of the GUI
    L = widgets.Layout(width='33%')
    inputColumn = widgets.VBox([topicInput, topicDescInput, goalInput, learningObjectiveInput,
                           difficultyInput, targetAudienceInput, nOptionsInput,
                           nCorrectInput, allocatedTimeInput], layout=L)  
    actionColumn = widgets.VBox([countInput, addButton, indexInput, removeButton], layout=L)  
    displayColumn = widgets.VBox([questionListOutput], layout=L)
    layout = widgets.HBox([inputColumn, actionColumn, displayColumn])

    # display GUI
    display(layout)

# Quiz generation

In [4]:
from openai import OpenAI
from openai.types.chat.completion_create_params import ResponseFormat
import json 


def promptBuilder(quizManager):
    questions = quizManager.questions
    
    topicDescriptions = '\n'.join(["The topic of "+q.topic+" is described as "+q.topicDesc+"." for q in questions])
    questionDescriptions = ';\n'.join([f"""{q.nOccurences} questions each with exactly {q.nOptions} options, {q.nCorrect} of which are correct options, with the rest of the options being distractors. 
    They should be about {q.topic}, these should target the following learning objective: {q.learningObjective}. 
    These questions should also be at the {q.goal} level in Bloom’s taxonomy, and should be suitable for {q.difficulty} level in {q.topic}, specially for {q.targetAudience} and should not take more than {q.allocatedTime} to answer.""" for q in questions])
    
    systemPrompt = f"""You are an expert quiz maker and especialist in {",".join([q.topic for q in questions])} for the purposes of learning support. 
    {topicDescriptions}
    
    Your task is focused on creating top quality multiple-choice question assessments. A multiple-choice question is a collection of three components (Stem, Correct Answers, Distractors), given a particular context of what the student is expected to know. The topic, as well as the context of the topic, will be provided in order to generate effective multiple-choice questions. 
    
    The stem refers to the question the student will attempt to answer, as well as the relevant context necessary in order to answer the question. It may be in the form of a question, an incomplete statement, or a scenario. The stem should focus on assessing the specific knowledge or concept the question aims to evaluate. 
    
    The Correct Answer(s) refers to the correct, undisputable answer(s) to the question in the stem. 
    
    A Distractor is an incorrect answer to the question in the stem and adheres to the following properties. 
    (1) A distractor should not be obviously wrong. In other words, it must still bear relations to the stem and correct answer. 
    (2) A distractor should be phrased positively and be a true statement that does not correctly answer the stem, all while giving no clues towards the correct answer. 
    (3) Although a distractor is incorrect, it must be plausible.
    (4) A distractor must be incorrect. It cannot be correct, or interpreted as correct by someone who strongly grasps the topic. 
    
    Use “None of the Above” or “All of the Above” style answer choices sparingly. These answer choices have been shown to, in general, be less effective at measuring or assessing student understanding. 
    
    Multiple-choice questions should be clear, concise, and grammatically correct statements. Make sure the questions are worded in a way that is easy to understand and does not introduce unnecessary complexity or ambiguity. Students should be able to understand the questions without confusion. The question should not be too long, and allow most students to finish in less than the given time. This means adhering to the following properties. 
    (1) Avoid using overly long sentences. 
    (2) If you refer to the same item or activity multiple times, use the same phrase each time. 
    (3) Ensure that each multiple-choice question provides full context. In other words, if a phrase or action is not part of the provided topic or topic context that a student is expected to know, then be sure to explain it briefly or consider not including it. 
    (4) Ensure that none of the distractors overlap. In other words, attempt to make each distractor reflect a different misconception on the topic, rather than a single one, if possible. 
    (5) Avoid too many clues. Do not include too many clues or hints in the answer options, which may make it too obvious for students to determine the correct answer. These options should require students to use their knowledge and reasoning to make an informed choice.
    
    Blooms’ Taxonomy and Action Verbs: 
    Multiple-choice questions must be well aligned to the learning objectives they are intended to assess students’ knowledge on. This implies that they must assess skills at the right cognitive level corresponding to the Bloom’s taxonomy categorization of the learning objective. Bloom’s Taxonomy offers a framework for categorizing the depth of learning, and it provides guidance on selecting appropriate action verbs when writing learning objectives. Here are the six levels of Bloom’s taxonomy and their definitions: 
    • Remember - This level involves retrieving, recognizing, and recalling relevant knowledge from long-term memory. 
    • Understand - At this level, learners construct meaning from oral, written, and graphic messages through interpreting, exemplifying, classifying, summarizing, inferring, comparing, and explaining. 
    • Apply - This level requires learners to carry out or use a procedure through executing or implementing it. 
    • Analyze - At this level, learners break material into constituent parts, determine how the parts relate to one another and to an overall structure or purpose through differentiating, organizing.
    • Evaluate - This level involves making judgments based on criteria and standards through checking and critiquing.
    • Create - At this level, learners put elements together to form a coherent or functional whole, or they reorganize elements into a new pattern or structure through generating.
    
    Difficulty levels:
    Multiple-choice questions must be obey certain rules to make sure the difficulty of the MCQ is appropriate:
    • Beginner - The question should be simple, the correct answer(s) should be obvious and the distractors should be easy to distinguish from the correct answer(s).
    • Intermediate - The question should be complicated, the correct answer(s) shouldn't pose too much of a problem to figure out but the distractors should make the student second guess their choices.
    • Advanced - The question should be complex, the correct answer(s) should be impossible to get right without good knowledge of the topic and the distractors should guessing impossible and disencourage uninformed choices.
    
    Output Format
    Output your multiple-choice quiz in an easy-to-parse json dictionary format. The quiz generated should have exactly {len(questions)} questions in total. 
    The questions generated should be the following:
    {questionDescriptions}
    
    Your return should be the exact json structure of the following example (if there was 1 question with 3 options, another with 2 options, and a final one with 4 options):
    {{
	"title": str,
	"questions": [
		{{
			"context": str,
			"question": str,
			"options": 
				[
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}},
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}},
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}}					
				]
		}},
		{{
			"context": str,
			"question": str,
			"options": 
				[
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}},
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}}				
				]
		}},
		{{
			"context": str,
			"question": str,
			"options": 
				[
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}},
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}},
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}},
					{{
						"option": str,
						"correct": boolean,
						"feedback": str
					}}					
				]
		}}
	]		
}}
    
    Below are some examples:
    Example 1 (
    - 2 questions each with exactly 4 options, 1 of which are correct options, with the rest of the options being distractors. 
    They should be about Generative AI and LLMs, these should target the following learning objective: Teaching about the topic, not testing knowledge of skills. 
    These questions should also be at the Remember level in Bloom’s taxonomy, and should be suitable for Beginner level in Generative AI and LLMs, specially for adults who have little to no knowledge about technologies and AI and should not take more than 1 minute to answer.;
    - 1 questions each with exactly 4 options, 2 of which are correct options, with the rest of the options being distractors. 
    They should be about Generative AI and LLMs, these should target the following learning objective: Teaching about the topic, not testing knowledge of skills. 
    These questions should also be at the Remember level in Bloom’s taxonomy, and should be suitable for Beginner level in Generative AI and LLMs, specially for adults who have little to no knowledge about technologies and AI and should not take more than 1 minute to answer.) : 
    
    {{
	"title": "MCQ about AI for Beginner's",
	"questions": [
		{{
			"context": "AI, GPT, and LLM are often used interchangeably nowadays.",
			"question": "What does "LLM" stand for in the context of AI?",
			"options":
				[
					{{
						"option": "Large Language Model",
						"correct": true,
						"feedback": "LLMs are trained on huge sets of data, so they are "large". They are a computer program trying to immitate (or "model") human "language" generation and processing."
					}},
					{{
						"option": "Long Local Machine",
						"correct": false,
						"feedback": "Good try, but while LLMs are a piece of technology, they are not physical machines."
					}},
					{{
						"option": "Long-term Learning Module",
						"correct": false,
						"feedback": "Good try, LLMs do indeed learn and are planned to last do so continuously for a long time, what distinguises LLMs is their size and natural language capabilities."
					}},
					{{
						"option": "Limited Liability Management",
						"correct": false,
						"feedback": "Fortunately, AIs and LLMs have nothing to do with coorporate companies... for now."
					}}						
				]
		}},
		{{
			"context": "There's been a lot of talk in the media about the impact of using Generative AI in nefarious ways."
			"question": "Why is it important to use Generative AI responsibly?",
			"options": 
				[
					{{
						"option": "To avoid spreading misinformation.",
						"correct": true,
						"feedback": "Generative AI models can generate convincing text that could be mistaken for factual information, so it's important to fact check anything generated by AIs!"
					}},
					{{
						"option": "To ensure it does not replace human jobs.",
						"correct": false,
						"feedback": "While concerns about AI and automation affecting employment exist, the primary reason for using Generative AI responsibly is not specifically about job replacement. It's more about ethical use, accuracy, and the potential impact on society, such as spreading misinformation or ethical concerns in its applications."
					}},
					{{
						"option": "To make sure it can only play video games.",
						"correct": false,
						"feedback": "The scope of Generative AI extends far beyond just playing video games. The importance of using Generative AI responsibly relates to its broader applications, including content creation, decision-making support, and more. The focus on responsible use is about preventing misuse and ensuring ethical considerations in its diverse applications, not limiting it to entertainment purposes."
					}},
					{{
						"option": "To prevent it from becoming too powerful.",
						"correct": false,
						"feedback": "The notion of AI becoming "too powerful" is a speculative and sci-fi scenario. The concern in the real world focuses on ensuring that AI is developed and used in ways that are ethical, fair, and do not harm society, rather than a fear of AI gaining autonomous power or control."
					}}					
				]
		}},
        {{
			"context":""
			"question": "Which of the following is an example of Generative AI's capabilities?",
			"options": 
				[
					{{
						"option": "Generating a news article based on a headline.",
						"correct": true,
						"feedback": "Generative AI can analyze the context and content implied by a headline and then produce a comprehensive news article that aligns with the style, tone, and factual requirements suggested by that headline. This capability demonstrates its ability to understand and generate contextually relevant text."
					}},
					{{
						"option": "Creating realistic video game environment art.",
						"correct": true,
						"feedback": "Generative AI can learn from vast amounts of data on landscapes, architectural styles, and environmental elements to create new, realistic video game environments. This process involves understanding the principles of design and environmental coherence to generate visually appealing and contextually suitable game worlds."
					}},
					{{
						"option": "Solving mathematical equations.",
						"correct": false,
						"feedback": " Solving mathematical equations typically involves computational and algorithmic approaches rather than generative processes. Generative AI focuses on creating new content based on learned patterns rather than solving structured, rule-based problems."
					}},
					{{
						"option": "Running physical simulations for engineering projects.",
						"correct": false,
						"feedback": "Running physical simulations involves computational models that predict how physical systems behave under various conditions, which is more about calculation and analysis rather than generating new, creative content. This task is typically handled by specialized simulation software, not generative AI."
					}}	
				]
		}},
  ]
		
}}

    Example 2 (
    - 2 questions each with exactly 3 options, 1 of which are correct options, with the rest of the options being distractors. 
    They should be about French History during the Napoleonic Wars, these should target the following learning objective: Testing basic french history knowledge. 
    These questions should also be at the Remember level in Bloom’s taxonomy, and should be suitable for Intermediate level in French History during the Napoleonic Wars, specially for high school students who have studied a history class chapter on French History and should not take more than 1 minute to answer.;
    - 1 questions each with exactly 3 options, 1 of which are correct options, with the rest of the options being distractors. 
    They should be about women in french history, these should target the following learning objective: Teach about important french female historical figures.
    These questions should also be at the Understand level in Bloom’s taxonomy, and should be suitable for Beginner level in women in french history, specially for high school students who have studied a history class chapter on French History and should not take more than 1 minute to answer.) : 
    - 1 questions each with exactly 3 options, 1 of which are correct options, with the rest of the options being distractors. 
    They should be about historical French landmarks, these should target the following learning objective: Test knowledge about the eiffel tower and other french monuments.
    These questions should also be at the Understand level in Bloom’s taxonomy, and should be suitable for Beginner level in historical French landmarks, specially for high school students who have studied a history class chapter on French History and should not take more than 1 minute to answer.) : 

{{
	"title": "French History: Napoleon and other historical figures",
	"questions": [
		{{
			"context": "",
			"question": "Which battle was Napoleon I’s final defeat?",
			"options": 
				[
					{{
						"option": "Battle of Waterloo",
						"correct": true,
						"feedback": "The Battle of Waterloo (June 18, 1815) was Napoleon I's final defeat, ending 23 years of recurrent warfare between France and the other powers of Europe. It was fought between Napoleon's 72,000 troops and the combined forces of the duke of Wellington's allied army of 68,000 (with British, Dutch, Belgian, and German units) and about 45,000 Prussians, the main force of Gebhard Leberecht von Blücher's command. Four days later Napoleon abdicated for the second time."
					}},
					{{
						"option": "Battle of Agincourt",
						"correct": false,
						"feedback": "The Battle of Agincourt took place on October 25, 1415, and was a major English victory over the French in the Hundred Years' War. This battle occurred nearly 400 years before Napoleon I's time and is notable for the use of the English longbow, which decimated the French knights and nobility. This makes it unrelated to Napoleon I's military campaigns and final defeat."
					}},
					{{
						"option": "Battle of Verdun",
						"correct": false,
						"feedback": "The Battle of Verdun, fought from February to December 1916 during World War I, was one of the longest and most devastating battles in world history. It involved French and German forces in a brutal conflict with enormous casualties on both sides. Since this battle took place nearly a century after Napoleon I's death, it could not represent his final defeat."
					}}						
				]
		}},
		{{
			"context": "",
			"question": "What French author’s father was a general for Napoleon and was nicknamed \“the Black Devil\”?",
			"options": 
				[
					{{
						"option": "Alexandre Dumas",
						"correct": true,
						"feedback": "Alexandre Dumas is well known for classics like The Three Musketeers, but his father Thomas-Alexandre Dumas was famous in his own right. The child of an enslaved Haitian and a white Frenchman, the elder Dumas joined the French army, rose through the ranks, and became France’s first Black general. The author Dumas is said to have based some of the action in his novels on his father’s exploits."
					}},
					{{
						"option": "Victor Hugo",
						"correct": false,
						"feedback": "Victor Hugo, the illustrious author of Les Misérables and The Hunchback of Notre-Dame, had a father who served as a high-ranking officer under Napoleon. However, Hugo's father, Joseph Léopold Sigisbert Hugo, was not known as "the Black Devil." Instead, Hugo's works often reflect his complex views on society, justice, and humanity, rather than direct inspiration from his father's military career. This distinguishes him from Alexandre Dumas, whose father's legendary military exploits directly influenced his storytelling."
					}},
					{{
						"option": "Albert Camus",
						"correct": false,
						"feedback": "Albert Camus, a philosopher and writer known for his contributions to absurdism and existentialism, was born in Algeria to a French-Algerian (Pied-Noir) family. His father, Lucien Camus, died in World War I, long after Napoleon's era, and had no historical ties to Napoleon's military campaigns. Camus is celebrated for works like The Stranger and The Plague, which explore the human condition and morality, unrelated to the Napoleonic military legacy."
					}}						
				]
		}},
		{{
			"context": "",
			"question": "Which of these French women was charged with the crime of wearing men’s clothing?",
			"options": 
				[
					{{
						"option": "Joan of Arc",
						"correct": true,
						"feedback": "Joan of Arc was a peasant girl who became a great military leader for France, defeating the English at Orléans in 1429. Unfortunately, she ran afoul of religious authorities by claiming God spoke directly to her (undermining the church) and wearing men’s clothing. She was convicted of heresy and burned at the stake. Decades later the conviction was overturned. In the 20th century she was made a saint."
					}},
					{{
						"option": "Marie-Antoinette",
						"correct": false,
						"feedback": "Marie-Antoinette, the last Queen of France before the French Revolution, was known for her extravagant lifestyle and the famous misquote "Let them eat cake." However, she was never charged with the crime of wearing men’s clothing. Her charges during her trial in 1793 were related to treason, depletion of the national treasury, and conspiracy against the security of the state, not her attire. This makes her an incorrect choice for this question."
					}},
					{{
						"option": "Marie Curie",
						"correct": false,
						"feedback": "Marie Curie was a renowned physicist and chemist, famous for her research on radioactivity and as the first woman to win a Nobel Prize. At no point was Marie Curie charged with the crime of wearing men’s clothing. Her professional and personal life was scrutinized for her scientific contributions and personal relationships, not her fashion choices."
					}}						
				]
		}},
		{{
			"context":"",
			"question": "Which of these French landmarks was designed to be taken down after 20 years?",
			"options": 
				[
					{{
						"option": "Eiffel Tower.",
						"correct": true,
						"feedback": "The Eiffel Tower was constructed for the International Exposition of 1889. Paris gave Gustave Eiffel use of the land the tower stood on for 20 years. Fortunately, the structure was able to prove its usefulness as an antenna in the blossoming field of radio, and in 1910 the lease was renewed for 70 years."
					}},
					{{
						"option": "Louvre Museum.",
						"correct": false,
						"feedback": "Originally a 12th-century fortress, the Louvre was never intended to be temporary. It evolved into a world-renowned museum, housing iconic art like the Mona Lisa, showcasing its permanent significance in French heritage."
					}},
					{{
						"option": "Arc de Triomphe.",
						"correct": false,
						"feedback": "Commissioned by Napoleon to honor military achievements, the Arc de Triomphe was completed in 1836 and designed as a permanent monument, not a temporary structure."
					}}	
				]
		}}
	]	
}}
    
    """  

    userPrompt = f"""Generate a top quality quiz with {len(questions)} multiple-choice questions that follow this:
    {questionDescriptions}
    """
    
    return (systemPrompt, userPrompt)

# sends prompt to the OpenAI API with the prompt from the promptBuilder
# and writes the .json file into the specified file.
# @params: 
# - promptBuild: a tuple (system prompt, user prompt)
# - fileNameToWrite: the name of the file into which to write the ChatGPT output
def generateQuiz(promptBuild, fileNameToWrite):
    client = OpenAI()
    # tell the API to output a valid JSON file as output
    responseFormat = ResponseFormat(type="json_object")
    
    completion = client.chat.completions.create(
    model="gpt-4-1106-preview",
    response_format=responseFormat,
    messages=[
            {"role": "system", "content": promptBuild[0]},
            {"role": "user", "content": promptBuild[1]}
        ]
    )
    output = completion.choices[0].message
    content = output.content
    
    # write the output onto the specified file
    with open(fileNameToWrite, 'w') as file:
        file.write(content)
    
    

# Quiz taking

In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import json

# loads a JSON file and returns a corresponding dictionary
def loadQuiz(filename):
    with open(filename, 'r') as file:
        quizData = json.load(file)
    return quizData

# displays the quiz dictionary with Jupyter widgets
# and displays feeeback upon submission
def displayQuiz(quizData):
    # creating the Jupyter widgets
    submitButton = widgets.Button(description="Submit Quiz")
    questionsOutput = widgets.Output()
    feedbackOutput = widgets.Output()

    # a list of tuples of (widget list, option list, question text) for each question
    questionsWidgets = []
    # for each question
    for question in quizData['questions']:
        with questionsOutput:
            # print the question and its context
            print("\n"+question['context'])#+"\n"
            print(question['question'])
            # create a list of checkbox widgets representing the options of the question
            options = [widgets.Checkbox(description=option['option'], 
                                        value=False, indent=False) for option in question['options']]
            # add a tuple to the list for later use
            questionsWidgets.append((options,
                                     question['options'],
                                     question['question']))
            # display the options of the question
            for option in options:
                display(option)

    # callback function for when the submit button is clicked
    def onSubmit(b):
        with feedbackOutput:
            clear_output()
            # feedback text container
            feedbackHtml = ""
            # for each question
            for optionWidgets, questionData, questionText in questionsWidgets:
                # write the question text to the feecback container
                feedbackHtml += f"<div><strong>{question_text}</strong></div>"
                # for each option
                for optionWidget, optionData in zip(optionWidgets, questionData):
                    # if the option is checked...
                    if optionWidget.value:
                        # and is a correct answer
                        if optionData['correct']:
                            # write the feedback explaining why it is correct (coloured green)
                            feedbackHtml += f"<div style='color: green; font-weight: bold;'>Selected: 
                                            {optionData['option']} - Feedback: {optionData['feedback']}</div>"
                        # and is an incorrect answer
                        else:
                            # write the feedback explaining why it is incorrect (coloured red)
                            feedbackHtml += f"<div style='color: red; font-weight: bold;'>Selected: 
                                            {optionData['option']} - Feedback: {optionData['feedback']}</div>"
                    # if the option isn't checked but is a correct answer
                    elif optionData['correct']:
                        # write the feedback why it is correct 
                        # (coloured orange to signify it was a missed correct answer)
                        feedbackHtml += f"<div style='color: orange; font-weight: bold;'>Missed: 
                                        {optionData['option']} - Feedback: {optionData['feedback']}</div>"
            # display the feedback text container
            display(HTML(feedbackHtml))
            # disable all widgets of the quiz (the quiz is submitted, so it's over)
            submitButton.disabled = True
            for options, _, _ in questionsWidgets:
                for option in options:
                    option.disabled = True
                    
    # add callback function to button
    submitButton.on_click(onSubmit)
    # display the quiz
    display(questionsOutput)
    display(submitButton)
    display(feedbackOutput)
